# 이력서 JSON 파서

PDF에서 추출된 JSON 데이터를 파싱하여 구조화된 이력서 정보를 추출합니다.

## 추출 정보
- ✅ 기본 정보 (이름, 직책)
- ✅ 연락처 정보 (이메일, 전화번호, GitHub, LinkedIn, 블로그)
- ✅ 자기소개
- ✅ 경력 정보 (회사, 직책, 기간, 업무, 프로젝트)
- ✅ 학력 정보 (학교, 학위, 전공, GPA)
- ✅ 출판물
- ✅ 채널/기술

## 1. 필요한 라이브러리 임포트

In [1]:
import json
import re
from typing import Dict, List, Optional, Any
from pathlib import Path
from datetime import datetime

## 2. ResumeParser 클래스 정의

In [2]:
class ResumeParser:
    """PDF에서 추출된 JSON 데이터를 파싱하여 구조화된 이력서 정보를 추출"""
    
    def __init__(self, json_path: str):
        self.json_path = Path(json_path)
        self.data = None
        self.elements = []
        
    def load_json(self) -> bool:
        """JSON 파일 로드"""
        try:
            with open(self.json_path, 'r', encoding='utf-8') as f:
                self.data = json.load(f)
                self.elements = self.data.get('elements', [])
            return True
        except Exception as e:
            print(f"Error loading JSON: {e}")
            return False
    
    def parse(self) -> Dict[str, Any]:
        """전체 이력서 파싱"""
        if not self.load_json():
            return {}
        
        result = {
            'basic_info': self._extract_basic_info(),
            'contact_info': self._extract_contact_info(),
            'introduction': self._extract_introduction(),
            'work_experiences': self._extract_work_experiences(),
            'education': self._extract_education(),
            'projects': self._extract_projects(),
            'publications': self._extract_publications(),
            'certificates': self._extract_certificates(),
            'awards': self._extract_awards(),
            'skills_and_channels': self._extract_skills_and_channels(),
        }
        
        return result
    
    def _get_text_by_category(self, category: str) -> List[Dict]:
        """특정 카테고리의 요소들 반환"""
        return [elem for elem in self.elements if elem.get('category') == category]
    
    def _extract_basic_info(self) -> Dict[str, Optional[str]]:
        """기본 정보 추출 (이름)"""
        basic_info = {
            'name': None,
            'title': None,
        }
        
        # 첫 번째 heading1에서 이름 추출 시도
        headings = self._get_text_by_category('heading1')
        if headings:
            first_heading = headings[0]['content']['text']
            
            # 한국어 이력서: "이력서 - ML Engineer / 최재강" 형식
            if '이력서' in first_heading or 'Resume' in first_heading:
                # "/" 기준으로 분리
                if '/' in first_heading:
                    parts = first_heading.split('/')
                    if len(parts) >= 2:
                        basic_info['name'] = parts[-1].strip()
                        basic_info['title'] = parts[-2].split('-')[-1].strip() if '-' in parts[-2] else None
            else:
                # 영어 이력서: 이름만 있는 경우
                basic_info['name'] = first_heading.strip()
        
        return basic_info
    
    def _extract_contact_info(self) -> Dict[str, Optional[str]]:
        """연락처 정보 추출"""
        contact_info = {
            'email': None,
            'phone': None,
            'address': None,
            'github': None,
            'linkedin': None,
            'blog': None,
            'portfolio': None,
        }
        
        # 모든 텍스트를 검사
        for elem in self.elements:
            text = elem['content']['text']
            
            # 이메일 추출 (공백이 포함된 경우도 처리)
            # "jdrummond@cs · toronto . edu" 같은 형식 처리
            email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
            email_match = re.search(email_pattern, text)
            if email_match and not contact_info['email']:
                contact_info['email'] = email_match.group(0)
            elif 'Email:' in text or '@' in text:
                # 공백이 포함된 이메일 처리
                cleaned_text = text.replace(' · ', '.').replace(' . ', '.').replace(' ', '')
                email_match = re.search(email_pattern, cleaned_text)
                if email_match and not contact_info['email']:
                    contact_info['email'] = email_match.group(0)
            
            # 전화번호 추출 (한국, 국제 형식)
            phone_pattern = r'(\d{3}[-\s]?\d{4}[-\s]?\d{4}|\d{3}[-\s]?\d{3}[-\s]?\d{4})'
            phone_match = re.search(phone_pattern, text)
            if phone_match and not contact_info['phone']:
                contact_info['phone'] = phone_match.group(0)
            
            # URL 추출
            if 'github.com' in text.lower():
                github_pattern = r'https?://github\.com/[\w-]+'
                github_match = re.search(github_pattern, text)
                if github_match:
                    contact_info['github'] = github_match.group(0)
            
            if 'linkedin.com' in text.lower() or 'liinkedin.com' in text.lower():
                linkedin_pattern = r'https?://(?:www\.)?li+nkedin\.com/in/[\w-]+'
                linkedin_match = re.search(linkedin_pattern, text)
                if linkedin_match:
                    contact_info['linkedin'] = linkedin_match.group(0)
            
            if 'tistory.com' in text.lower() or 'blog' in text.lower():
                blog_pattern = r'https?://[\w-]+\.(?:tistory\.com|blog\.[\w.]+)'
                blog_match = re.search(blog_pattern, text)
                if blog_match:
                    contact_info['blog'] = blog_match.group(0)
            
            # 주소 추출 (영어 이력서)
            if 'Toronto' in text or 'Canada' in text or ('University' in text and 'Department' in text):
                if not contact_info['address'] and len(text) > 20 and len(text.split('\n')) > 2:
                    contact_info['address'] = text
        
        return contact_info
    
    def _extract_introduction(self) -> Optional[str]:
        """자기소개 추출"""
        introduction = None
        
        # "Introduction" 섹션 찾기
        for i, elem in enumerate(self.elements):
            if elem.get('category') == 'heading1' and 'Introduction' in elem['content']['text']:
                # 다음 요소가 list나 paragraph면 그것이 소개글
                if i + 1 < len(self.elements):
                    next_elem = self.elements[i + 1]
                    if next_elem.get('category') in ['list', 'paragraph']:
                        introduction = next_elem['content']['text']
                        break
        
        return introduction
    
    def _extract_work_experiences(self) -> List[Dict[str, Any]]:
        """경력 정보 추출"""
        experiences = []
        
        # "Experience" 또는 "Career" 섹션 찾기
        in_experience_section = False
        current_experience = None
        current_project_name = None
        
        for i, elem in enumerate(self.elements):
            text = elem['content']['text']
            category = elem.get('category')
            
            # 섹션 시작 감지 (Experience, Career, Research Experience, Teaching Experience)
            if category == 'heading1':
                # Experience 관련 섹션인지 확인
                is_experience_section = (
                    ('Experience' in text or 'Career' in text) and
                    not any(exclude in text for exclude in ['Publication', 'Award'])
                )
                
                if is_experience_section:
                    in_experience_section = True
                    continue
            
            # 다른 주요 섹션 시작되면 종료 (Education, Publications, Project 등)
            if category == 'heading1' and in_experience_section:
                if any(keyword in text for keyword in ['Education', 'Publication', 'Award', 'Working Papers', 'Project', 'Certification', 'Technical Skills']):
                    # 현재 경력 저장
                    if current_experience:
                        experiences.append(current_experience)
                    in_experience_section = False
                    current_experience = None
                    current_project_name = None
                # 회사명/프로젝트는 계속 처리
                else:
                    # 회사명/직책이 있는 heading1
                    if 'I' in text or '|' in text or any(word in text.lower() for word in ['engineer', 'developer', 'manager']):
                        if current_experience:
                            experiences.append(current_experience)
                        
                        # 회사명과 직책 분리
                        company = None
                        position = None
                        
                        if 'I' in text:
                            parts = text.split('I')
                            if len(parts) >= 2:
                                company = parts[0].strip()
                                position = parts[1].strip()
                        elif '|' in text:
                            parts = text.split('|')
                            if len(parts) >= 2:
                                company = parts[0].strip()
                                position = parts[1].strip()
                        else:
                            # 전체를 회사명으로
                            company = text.strip()
                        
                        current_experience = {
                            'company_name': company,
                            'position': position,
                            'start_date': None,
                            'end_date': None,
                            'description': '',
                            'achievements': '',
                            'projects': [],
                        }
                        current_project_name = None
                    # 프로젝트 이름인 경우 (회사 내 프로젝트)
                    elif current_experience:
                        # heading1 또는 paragraph 중 프로젝트 키워드 있는 경우  
                        is_project_title = (
                            (len(text) < 100) and  
                            ('개발' in text or '프로젝트' in text or '챗봇' in text or 'QA' in text or 
                             'POC' in text or '구현' in text or '최적화' in text) and
                            # 성과나 업무 설명 아닌 것
                            not ('성과' in text or '달성' in text or '향상' in text or '증가' in text)
                        )
                        
                        if is_project_title:
                            current_project_name = text.strip()
                            current_experience['projects'].append({
                                'name': current_project_name,
                                'description': '',
                            })
            
            if in_experience_section:
                # 먼저 paragraph가 프로젝트 제목인지 확인
                if category == 'paragraph' and current_experience:
                    # 원본 텍스트에서 bullet point 체크 (strip 전)
                    stripped = text.strip()
                    is_project_title = (
                        (len(text) < 100) and
                        # 프로젝트 관련 키워드 (한국어 + 영어)
                        ('개발' in text or '프로젝트' in text or 'project' in text.lower() or
                         '챗봇' in text or 'chatbot' in text.lower() or
                         'QA 홈페이지' in text or 'POC' in text or 'LogitProcessor 구현' in text or
                         'development' in text.lower() or 'system' in text.lower()) and
                        # 제외할 패턴들 (한국어 + 영어)
                        not ('성과' in text or 'achievement' in text.lower() or
                             '달성' in text or '향상' in text or '증가' in text or
                             '기존' in text or '목표' in text or '모델' in text or 
                             stripped.startswith('·') or stripped.startswith('。') or stripped.startswith('-') or
                             '최적화' == stripped or '고도화' in text or
                             '점수' in text or '진행' in text or '수주' in text)
                    )
                    
                    if is_project_title:
                        current_project_name = text.strip()
                        current_experience['projects'].append({
                            'name': current_project_name,
                            'description': '',
                        })
                        continue  # 다음 element로
                
                # Research Experience 섹션의 paragraph 형식 경력 정보 파싱 (영어 이력서)
                # 한국어 날짜 파싱 전에 먼저 체크
                if category == 'paragraph' and in_experience_section:
                    # 영어 날짜 패턴 (May 2016 to August 2016)
                    date_pattern_en = r'(January|February|March|April|May|June|July|August|September|October|November|December|Spring|Fall)\s+(\d{4})\s+to\s+(Present|(January|February|March|April|May|June|July|August|September|October|November|December|Spring|Fall)\s+(\d{4}))'
                    date_match_en = re.search(date_pattern_en, text)
                    
                    # 직책 패턴
                    position_patterns = [
                        r'^(Research (?:Intern|Assistant|Associate|Fellow|Scientist))',
                        r'^(Teaching Assistant|Teaching Fellow)',
                        r'^(Directed Study)',
                    ]
                    
                    position_match = None
                    for pattern in position_patterns:
                        position_match = re.search(pattern, text)
                        if position_match:
                            break
                    
                    # 새로운 경력 (직책 + 날짜)
                    if position_match and date_match_en:
                        if current_experience:
                            experiences.append(current_experience)
                        
                        position = position_match.group(1)
                        
                        # 조직명 추출
                        company = None
                        parts = text.split(',')
                        if len(parts) >= 2:
                            company = parts[1].strip()
                            if ' with ' in company:
                                company = company.split(' with ')[0].strip()
                        
                        # 날짜 파싱
                        start_date = f"{date_match_en.group(1)} {date_match_en.group(2)}"
                        end_str = date_match_en.group(3)
                        end_date = "Present" if "Present" in end_str else end_str
                        
                        current_experience = {
                            'company_name': company,
                            'position': position,
                            'start_date': start_date,
                            'end_date': end_date,
                            'description': '',
                            'achievements': '',
                            'projects': [],
                        }
                        current_project_name = None
                        continue
                
                # 날짜 정보 추출 (오른쪽에 위치한 날짜만 - x > 0.5, 한국어 형식)
                if category == 'paragraph' and re.search(r'\d{4}', text):
                    # "2023.08 - Present" 또는 "2023.08 - 현재" 형식
                    date_pattern = r'(\d{4}\.\d{2}|\d{4})[\s-]+(Present|present|현재|\d{4}\.\d{2}|\d{4})'
                    date_match = re.search(date_pattern, text)
                    # 날짜가 오른쪽에 위치하는지 확인 (x좌표 > 0.5)
                    coords = elem.get('coordinates', [])
                    is_right_aligned = coords and len(coords) > 0 and coords[0].get('x', 0) > 0.5
                    
                    if date_match and current_experience and is_right_aligned:
                        current_experience['start_date'] = date_match.group(1)
                        end = date_match.group(2)
                        current_experience['end_date'] = None if end.lower() in ['present', '현재'] else end
                
                # 업무 내용이나 성과
                elif category in ['paragraph', 'list']:
                    content = text.strip()
                    
                    # Research Experience 섹션의 paragraph 형식 경력 정보 파싱
                    # 예: "Research Intern, Microsoft Research, with Ian Kash and Peter Key, May 2016 to August 2016."
                    if category == 'paragraph' and in_experience_section:
                        # 날짜 패턴 확인 (May 2016 to August 2016 형식)
                        date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December|Spring|Fall)\s+(\d{4})\s+to\s+(Present|(January|February|March|April|May|June|July|August|September|October|November|December|Spring|Fall)\s+(\d{4}))'
                        date_match = re.search(date_pattern, text)
                        
                        # 직책 패턴 (Research Intern, Research Assistant, etc.)
                        position_patterns = [
                            r'^(Research (?:Intern|Assistant|Associate|Fellow|Scientist))',
                            r'^(Teaching Assistant|Teaching Fellow)',
                            r'^((?:Graduate|Undergraduate) Research Assistant)',
                            r'^((?:Postdoctoral|Post-doctoral) (?:Researcher|Fellow))',
                        ]
                        
                        position_match = None
                        for pattern in position_patterns:
                            position_match = re.search(pattern, text)
                            if position_match:
                                break
                        
                        # 새로운 경력 항목인 경우 (직책과 날짜가 있음)
                        if position_match and date_match:
                            # 이전 경력 저장
                            if current_experience:
                                experiences.append(current_experience)
                            
                            position = position_match.group(1)
                            
                            # 조직명 추출 (직책 다음 콤마로 구분)
                            company = None
                            parts = text.split(',')
                            if len(parts) >= 2:
                                company = parts[1].strip()
                                # "with" 이후는 멘토/지도교수이므로 제외
                                if ' with ' in company:
                                    company = company.split(' with ')[0].strip()
                            
                            # 날짜 파싱
                            start_month = date_match.group(1)
                            start_year = date_match.group(2)
                            end_str = date_match.group(3)
                            
                            start_date = f"{start_month} {start_year}"
                            end_date = "Present" if "Present" in end_str else end_str
                            
                            current_experience = {
                                'company_name': company,
                                'position': position,
                                'start_date': start_date,
                                'end_date': end_date,
                                'description': '',
                                'achievements': '',
                                'projects': [],
                            }
                            current_project_name = None
                            continue
                    
                    # 성과 섹션 (한국어 + 영어)
                    if '성과' in text or 'achievement' in text.lower() or 'accomplishment' in text.lower():
                        if current_experience and len(content) <20:
                            # 성과 제목만 있음
                            pass
                        elif current_experience:
                            current_experience['achievements'] += content + '\n'
                    # 프로젝트 설명
                    elif current_project_name and current_experience and len(current_experience['projects']) > 0:
                        current_experience['projects'][-1]['description'] += content + '\n'
                    
                    # 일반 업무 내용
                    elif current_experience:
                        # 너무 짧은 제목은 제외
                        if len(content) > 10:
                            current_experience['description'] += content + '\n'
        
        # 마지막 경력 저장
        if current_experience:
            experiences.append(current_experience)
        
        return experiences
    
    def _extract_education(self) -> List[Dict[str, Any]]:
        """학력 정보 추출"""
        education_list = []
        
        in_education_section = False
        current_education = None
        
        for i, elem in enumerate(self.elements):
            text = elem['content']['text']
            category = elem.get('category')
            
            # Education 섹션 시작
            if category == 'heading1' and 'Education' in text:
                in_education_section = True
                continue
            
            # 다른 섹션 시작되면 종료
            if category == 'heading1' and in_education_section and 'Education' not in text:
                if current_education:
                    education_list.append(current_education)
                in_education_section = False
                current_education = None
            
            if in_education_section:
                # 학위 정보 파싱
                if category == 'paragraph':
                    # "PhD Computer Science, University of Toronto, (expected) Spring 2017." 형식
                    # "국민대학교 컴퓨터공학과 석사" 형식
                    
                    degree_match = re.search(r'(PhD|M\.S\.|B\.S\.|석사|학사|박사)', text)
                    if degree_match:
                        if current_education:
                            education_list.append(current_education)
                        
                        degree = degree_match.group(1)
                        institution = None
                        major = None
                        start_date = None
                        end_date = None
                        
                        # 대학명 추출
                        if 'University' in text:
                            uni_match = re.search(r'(University of [\w\s]+|[\w\s]+University)', text)
                            if uni_match:
                                institution = uni_match.group(1).strip(',').strip()
                        elif '대학교' in text:
                            uni_match = re.search(r'([\w]+대학교)', text)
                            if uni_match:
                                institution = uni_match.group(1)
                        
                        # 전공 추출
                        if 'Computer Science' in text:
                            major = 'Computer Science'
                        elif 'Mathematics' in text:
                            major = 'Mathematics'
                        elif '컴퓨터공학' in text:
                            major = '컴퓨터공학'
                        elif '소프트웨어' in text:
                            major = '소프트웨어'
                        
                        # 날짜 추출
                        date_pattern = r'(Spring|Fall|December|Spring)?\s*(\d{4})'
                        dates = re.findall(date_pattern, text)
                        if dates:
                            if len(dates) >= 1:
                                end_date = dates[-1][1] if dates[-1][1] else dates[-1][0]
                        
                        current_education = {
                            'institution': institution,
                            'degree': degree,
                            'major': major,
                            'minor': None,
                            'start_date': start_date,
                            'end_date': end_date,
                            'gpa': None,
                            'honors': '',
                            'relevant_courses': '',
                        }
                    
                    # GPA 추출
                    elif 'GPA' in text and current_education:
                        gpa_match = re.search(r'GPA:\s*([\d.]+)', text)
                        if gpa_match:
                            current_education['gpa'] = gpa_match.group(1)
                    
                    # 상세 정보
                    elif current_education:
                        if 'Advisor' in text or 'Co-advisors' in text:
                            current_education['honors'] += text + '\n'
                        elif 'Relevant Courses' in text or 'Courses' in text:
                            current_education['relevant_courses'] += text + '\n'
                        elif 'Honors' in text or 'Dean' in text:
                            current_education['honors'] += text + '\n'
                        elif 'Minor' in text:
                            minor_match = re.search(r'Minor:\s*([\w\s]+)', text)
                            if minor_match:
                                current_education['minor'] = minor_match.group(1).strip('.')
        
        # 마지막 학력 저장
        if current_education:
            education_list.append(current_education)
        
        # Career 섹션의 학력 정보도 추출 (한국어 이력서)
        for elem in self.elements:
            text = elem['content']['text']
            if elem.get('category') == 'list':
                if '대학교' in text and ('석사' in text or '학사' in text):
                    parts = text.strip('· ').strip()
                    
                    # 이미 추가되지 않았다면 추가 (학위+전공 조합으로 중복 체크)
                    degree = '석사' if '석사' in parts else '학사'
                    institution = re.search(r'([\w]+대학교)', parts)
                    
                    # 전공을 더 세밀하게 추출
                    if '소프트웨어학부' in parts:
                        major_text = '소프트웨어학부'
                    elif '컴퓨터공학과' in parts:
                        major_text = '컴퓨터공학과'
                    elif '소프트웨어' in parts:
                        major_text = '소프트웨어'
                    elif '컴퓨터공학' in parts:
                        major_text = '컴퓨터공학'
                    else:
                        major_text = None
                    
                    # 같은 학위+전공 조합이 없으면 추가
                    inst_name = institution.group(1) if institution else None
                    is_duplicate = any(
                        edu['institution'] == inst_name and 
                        edu['degree'] == degree and 
                        edu['major'] == major_text 
                        for edu in education_list
                    )
                    
                    if not is_duplicate:
                        education_list.append({
                            'institution': inst_name,
                            'degree': degree,
                            'major': major_text,
                            'minor': None,
                            'start_date': None,
                            'end_date': None,
                            'gpa': None,
                            'honors': '',
                            'relevant_courses': '',
                        })
        
        return education_list
    
    def _extract_projects(self) -> List[Dict[str, Any]]:
        """프로젝트 정보 추출 (별도 Project 섹션의 외부 프로젝트)"""
        projects = []
        
        in_project_section = False
        current_project = None
        
        for elem in self.elements:
            text = elem['content']['text']
            category = elem.get('category')
            
            # Project 섹션 시작
            if category == 'heading1' and text.strip() == 'Project':
                in_project_section = True
                continue
            
            # 다른 주요 섹션 시작되면 종료
            if category == 'heading1' and in_project_section:
                if any(keyword in text for keyword in ['Certification', 'Award', 'Publication', 'Education']):
                    if current_project:
                        projects.append(current_project)
                    in_project_section = False
                    current_project = None
                    continue
                
                # Project 섹션 내의 프로젝트 제목 (heading1)
                # 예: "가짜연구소 러너 활동", "개인 프로젝트" 등
                if current_project:
                    projects.append(current_project)
                
                current_project = {
                    'title': text.strip(),
                    'description': '',
                    'start_date': None,
                    'end_date': None,
                }
            
            # 프로젝트 섹션 내의 내용
            elif in_project_section:
                if category == 'paragraph':
                    # 날짜 패턴 확인
                    date_pattern = r'(\d{4}\.\d{2})\s*-\s*(Present|현재)'
                    date_match = re.search(date_pattern, text)
                    if date_match and current_project:
                        current_project['start_date'] = date_match.group(1)
                        current_project['end_date'] = 'Present' if date_match.group(2) in ['Present', '현재'] else None
                    elif current_project:
                        # 프로젝트 제목일 수도 (paragraph인 경우)
                        if len(text) < 100 and len(current_project['description']) == 0:
                            current_project['description'] += text + '\n'
                        else:
                            current_project['description'] += text + '\n'
                elif category == 'list' and current_project:
                    current_project['description'] += text + '\n'
        
        if current_project:
            projects.append(current_project)
        
        return projects
    
    def _extract_publications(self) -> List[Dict[str, Any]]:
        """출판물/논문 정보 추출"""
        publications = []
        
        in_publication_section = False
        
        for i, elem in enumerate(self.elements):
            text = elem['content']['text']
            category = elem.get('category')
            
            # Publications 섹션 시작
            if category == 'heading1' and 'Publication' in text:
                in_publication_section = True
                continue
            
            # 다른 섹션 시작
            if category == 'heading1' and in_publication_section:
                if 'Publication' not in text and 'Working Papers' not in text:
                    in_publication_section = False
            
            if in_publication_section and category == 'paragraph':
                # 논문 제목, 저자, 학회 정보 파싱
                # "Title, Authors, Venue (Conference), pages."
                
                # 콤마로 분리
                parts = text.split(',')
                
                if len(parts) >= 2:
                    title = parts[0].strip()
                    authors = parts[1].strip() if len(parts) > 1 else ''
                    venue = ''
                    year = None
                    pages = None
                    
                    # Year 추출
                    year_match = re.search(r'\((\d{4})\)|\s(\d{4})', text)
                    if year_match:
                        year = year_match.group(1) or year_match.group(2)
                    
                    # Pages 추출
                    pages_match = re.search(r'pages?\s*([\d-]+)', text)
                    if pages_match:
                        pages = pages_match.group(1)
                    
                    # Venue (Proc. of ...)
                    venue_match = re.search(r'Proc\. of[^,]+', text)
                    if venue_match:
                        venue = venue_match.group(0)
                    
                    publications.append({
                        'title': title,
                        'authors': authors,
                        'venue': venue,
                        'year': year,
                        'pages': pages,
                        'description': text,
                    })
        
        return publications
    
    def _extract_skills_and_channels(self) -> List[Dict[str, Any]]:
        """기술/채널 정보 추출"""
        skills = []
        
        in_tech_section = False
        
        for i, elem in enumerate(self.elements):
            text = elem['content']['text']
            category = elem.get('category')
            
            # Technical Skills 섹션 시작
            if category == 'heading1' and ('Technical Skills' in text or 'Skills' in text):
                in_tech_section = True
                continue
            
            # 다른 섹션 시작되면 종료
            if category == 'heading1' and in_tech_section:
                in_tech_section = False
            
            # Technical Skills 섹션 내용 파싱
            if in_tech_section and category == 'paragraph':
                # 줄바꿈으로 분리
                lines = text.split('\n')
                for line in lines:
                    line = line.strip()
                    if not line:
                        continue
                    
                    # "Category: Details" 형식 파싱
                    if ':' in line:
                        parts = line.split(':', 1)
                        if len(parts) == 2:
                            category_name = parts[0].strip()
                            details = parts[1].strip()
                            skills.append({
                                'category': 'technical_skill',
                                'name': category_name,
                                'value': details
                            })
            
            # Channel 섹션
            if category == 'heading1' and 'Channel' in text:
                # 다음 list 항목들이 채널
                idx = self.elements.index(elem)
                if idx + 1 < len(self.elements):
                    next_elem = self.elements[idx + 1]
                    if next_elem.get('category') == 'list':
                        lines = next_elem['content']['text'].split('\n')
                        for line in lines:
                            if 'Github' in line:
                                skills.append({'category': 'channel', 'name': 'Github', 'value': line.split('I')[-1].strip() if 'I' in line else line})
                            elif 'Linkedin' in line or 'Linkedln' in line:
                                skills.append({'category': 'channel', 'name': 'LinkedIn', 'value': line.split('I')[-1].strip() if 'I' in line else line})
                            elif 'Blog' in line:
                                skills.append({'category': 'channel', 'name': 'Blog', 'value': line.split('I')[-1].strip() if 'I' in line else line})
        
        return skills
    
    def _extract_certificates(self) -> List[Dict[str, Any]]:
        """자격증 정보 추출"""
        certificates = []
        
        in_cert_section = False
        
        for elem in self.elements:
            text = elem['content']['text']
            category = elem.get('category')
            
            # Certification 섹션 시작
            if category == 'heading1' and 'Certification' in text:
                in_cert_section = True
                continue
            
            # 다른 섹션 시작되면 종료
            if category == 'heading1' and in_cert_section:
                in_cert_section = False
            
            # Certification 섹션 내의 자격증
            if in_cert_section and category == 'list':
                # 줄바꿈으로 분리된 자격증들
                lines = text.split('\n')
                for line in lines:
                    line = line.strip().strip('·').strip()
                    if line and len(line) > 2:
                        certificates.append({
                            'name': line,
                            'issuer': None,
                            'date': None,
                        })
        
        return certificates
    
    def _extract_awards(self) -> List[Dict[str, Any]]:
        """수상 이력 추출"""
        awards = []
        
        in_award_section = False
        
        for elem in self.elements:
            text = elem['content']['text']
            category = elem.get('category')
            
            # Award 섹션 시작
            if category == 'heading1' and 'Award' in text:
                in_award_section = True
                continue
            
            # 다른 섹션 시작되면 종료
            if category == 'heading1' and in_award_section:
                in_award_section = False
            
            # Award 섹션 내의 수상 내역
            if in_award_section and category in ['list', 'paragraph']:
                # 줄바꿈으로 분리된 수상 내역들
                lines = text.split('\n')
                for line in lines:
                    line = line.strip().strip('·').strip()
                    if line and len(line) > 2:
                        awards.append({
                            'name': line,
                            'issuer': None,
                            'date': None,
                        })
        
        return awards

## 3. 출력 함수 정의

In [3]:
def print_parsed_resume(parsed_data: Dict[str, Any], title: str = "Parsed Resume"):
    """파싱된 이력서 정보를 보기 좋게 출력"""
    print("\n" + "="*80)
    print(f" {title}")
    print("="*80)
    
    # 기본 정보
    print("\n📋 기본 정보:")
    basic = parsed_data.get('basic_info', {})
    for key, value in basic.items():
        if value:
            print(f"  • {key}: {value}")
    
    # 연락처 정보
    print("\n📞 연락처 정보:")
    contact = parsed_data.get('contact_info', {})
    for key, value in contact.items():
        if value:
            if key == 'address':
                print(f"  • {key}:")
                for line in value.split('\n'):
                    if line.strip():
                        print(f"      {line.strip()}")
            else:
                print(f"  • {key}: {value}")
    
    # 자기소개
    intro = parsed_data.get('introduction')
    if intro:
        print("\n💬 자기소개:")
        for line in intro.split('\n'):
            if line.strip():
                print(f"  {line.strip()}")
    
    # 경력
    experiences = parsed_data.get('work_experiences', [])
    if experiences:
        print(f"\n💼 경력 ({len(experiences)}개):")
        for i, exp in enumerate(experiences, 1):
            print(f" \n  [{i}] {exp.get('company_name', 'N/A')}")
            if exp.get('position'):
                print(f"      직책: {exp['position']}")
            if exp.get('start_date') or exp.get('end_date'):
                period = f"{exp.get('start_date', '')} ~ {exp.get('end_date') or '현재'}"
                print(f"      기간: {period}")
            if exp.get('description'):
                desc_lines = [line for line in exp['description'].strip().split('\n') if line.strip()]
                if desc_lines:
                    print(f"      업무 내용:")
                    for line in desc_lines[:5]:
                        cleaned = line.strip()
                        if cleaned:
                            print(f"        - {cleaned[:120]}")
            if exp.get('projects'):
                print(f"      프로젝트 ({len(exp['projects'])}개):")
                for j, proj in enumerate(exp['projects'], 1):
                    print(f"        [{j}] {proj.get('name', 'N/A')}")
                    proj_desc = proj.get('description', '').strip()
                    if proj_desc:
                        proj_lines = [line for line in proj_desc.split('\n') if line.strip()][:3]
                        for line in proj_lines:
                            print(f"            - {line.strip()[:100]}")
            if exp.get('achievements'):
                ach_lines = [line for line in exp['achievements'].strip().split('\n') if line.strip()]
                if ach_lines:
                    print(f"      성과:")
                    for line in ach_lines[:3]:
                        cleaned = line.strip()
                        if cleaned:
                            print(f"        - {cleaned[:120]}")
    
    # 학력
    education = parsed_data.get('education', [])
    if education:
        print(f"\n🎓 학력 ({len(education)}개):")
        for i, edu in enumerate(education, 1):
            print(f"\n  [{i}] {edu.get('institution', 'N/A')}")
            if edu.get('degree'):
                print(f"      학위: {edu['degree']}")
            if edu.get('major'):
                print(f"      전공: {edu['major']}")
            if edu.get('gpa'):
                print(f"      GPA: {edu['gpa']}")
            if edu.get('end_date'):
                print(f"      졸업: {edu['end_date']}")
    
    # 출판물
    publications = parsed_data.get('publications', [])
    if publications:
        print(f"\n📚 출판물 ({len(publications)}개):")
        for i, pub in enumerate(publications, 1):
            print(f"\n  [{i}] {pub.get('title', 'N/A')}")
            if pub.get('authors'):
                print(f"      저자: {pub['authors'][:80]}")
            if pub.get('venue'):
                print(f"      학회: {pub['venue'][:80]}")
            if pub.get('year'):
                print(f"      연도: {pub['year']}")
    
    # 채널/스킬
    skills = parsed_data.get('skills_and_channels', [])
    if skills:
        print(f"\n🔗 채널 및 기술 ({len(skills)}개):")
        for skill in skills:
            print(f"  • {skill.get('name', 'N/A')}: {skill.get('value', '')[:60]}")
    
    print("\n" + "="*80 + "\n")

## 4. 이력서 파싱 실행

### 4-1. 한국어 이력서 파싱 (cv_data00.json)

In [ ]:
# 한국어 이력서 파싱
parser_kr = ResumeParser('cv_data00.json')
result_kr = parser_kr.parse()

# 결과 출력
print_parsed_resume(result_kr, "한국어 이력서 (cv_data00.json)")

### 4-2. 영어 이력서 파싱 (cv_data01.json)

In [5]:
# 영어 이력서 파싱
parser_en = ResumeParser('cv_data01.json')
result_en = parser_en.parse()

# 결과 출력
print_parsed_resume(result_en, "영어 이력서 (cv_data01.json)")


 영어 이력서 (cv_data01.json)

📋 기본 정보:
  • name: Joanna Drummond

📞 연락처 정보:
  • email: jdrummond@cs.toronto.edu
  • address:
      Department of Computer Science
      University of Toronto
      10 King's College Road, Rm. 3302
      Toronto, Ontario, M5S 3G4 Canada

💼 경력 (6개):
 
  [1] Microsoft Research
      직책: Research Intern
      기간: May 2016 ~ August 2016
      업무 내용:
        - Investigated simple pricing for cloud computing.
 
  [2] University of Toronto
      직책: Research Assistant
      기간: August 2011 ~ December 2014
      업무 내용:
        - Investigating Bayes-Nash and ex-post equilibria for matching games with imperfect information.
        - Investigating stable and approximately stable matching using multi-attribute preference information.
        - Investigating elicitation schemes using multi-attribute based queries.
        - Investigating stable and approximately stable matching on social networks.
        - Investigated elicitation schemes for the stable matching proble

## 5. 파싱 결과 요약

In [6]:
# 두 이력서의 파싱 결과 요약
print("\n" + "="*80)
print("  📊 파싱 요약")
print("="*80)

results = {
    'cv_data00.json (한국어)': result_kr,
    'cv_data01.json (영어)': result_en
}

for filename, result in results.items():
    print(f"\n{filename}:")
    print(f"  - 이름: {result.get('basic_info', {}).get('name')}")
    print(f"  - 이메일: {result.get('contact_info', {}).get('email')}")
    print(f"  - 전화번호: {result.get('contact_info', {}).get('phone')}")
    print(f"  - 경력: {len(result.get('work_experiences', []))}개")
    print(f"  - 학력: {len(result.get('education', []))}개")
    print(f"  - 프로젝트: {len(result.get('projects', []))}개")
    print(f"  - 출판물: {len(result.get('publications', []))}개")

print("\n" + "="*80)


  📊 파싱 요약

cv_data00.json (한국어):
  - 이름: 최재강
  - 이메일: workd.official@gmail.com
  - 전화번호: 010-9040-6282
  - 경력: 1개
  - 학력: 2개
  - 프로젝트: 1개
  - 출판물: 0개

cv_data01.json (영어):
  - 이름: Joanna Drummond
  - 이메일: jdrummond@cs.toronto.edu
  - 전화번호: None
  - 경력: 6개
  - 학력: 3개
  - 프로젝트: 0개
  - 출판물: 3개



## 6. 결과를 JSON 파일로 저장 (선택사항)

In [7]:
# 파싱 결과를 JSON 파일로 저장
output_data = {
    'cv_data00.json': result_kr,
    'cv_data01.json': result_en
}

output_file = 'parsed_resumes_output.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"✅ 파싱 결과가 '{output_file}' 파일로 저장되었습니다.")

✅ 파싱 결과가 'parsed_resumes_output.json' 파일로 저장되었습니다.


## 7. 개별 필드 접근 예제

In [8]:
# 특정 정보만 추출하는 예제

# 이름과 이메일만 추출
print("\n=== 기본 정보 추출 ===")
print(f"한국어 이력서 - 이름: {result_kr['basic_info']['name']}")
print(f"한국어 이력서 - 이메일: {result_kr['contact_info']['email']}")
print(f"영어 이력서 - 이름: {result_en['basic_info']['name']}")
print(f"영어 이력서 - 이메일: {result_en['contact_info']['email']}")

# 경력 정보 접근
print("\n=== 경력 정보 ===")
for exp in result_kr['work_experiences']:
    print(f"회사: {exp['company_name']}")
    print(f"직책: {exp['position']}")
    print(f"기간: {exp['start_date']} ~ {exp['end_date'] or '현재'}")
    if exp['projects']:
        print(f"프로젝트: {len(exp['projects'])}개")
        for proj in exp['projects']:
            print(f"  - {proj['name']}")

# 학력 정보 접근
print("\n=== 학력 정보 ===")
for edu in result_en['education']:
    print(f"{edu['degree']} - {edu['major']} (GPA: {edu['gpa']})")


=== 기본 정보 추출 ===
한국어 이력서 - 이름: 최재강
한국어 이력서 - 이메일: workd.official@gmail.com
영어 이력서 - 이름: Joanna Drummond
영어 이력서 - 이메일: jdrummond@cs.toronto.edu

=== 경력 정보 ===
회사: 오케스트로
직책: ML Engineer
기간: 2023.08 ~ 현재
프로젝트: 4개
  - 회사 QA 홈페이지 챗봇 개발
  - 한국행정연구원 보고서 목차별 요약 POC
  - · 요약 파이프라인 추론 속도 최적화
  - LLM 외국어 Block LogitProcessor 구현

=== 학력 정보 ===
PhD - Computer Science (GPA: 3.83)
M.S. - Computer Science (GPA: 3.93)
B.S. - Computer Science (GPA: 3.73)
